In [4]:
def gen_data_vec_btc(t, y_price, x, LTV0, bs_vol, Growth_rate, Real_vol): 
    
    """
        Purpose: Simulate a CRP performance during a loan life term   
            - x: collateral token (BTC)
            - y: loan token (USDC)
        Assumption: 
            1. Rebalnce following a delta replicating strategy of a call option on Token / Collateral. 
            2. Price percentage change (approximately equals to log(p(t+1)/p(t))) follows a linear growth treand 
               (slope = growth rate) plus a random walk Brownie motion. 
        Input: 
            - t: term of the loan 
            - y_price: price of loan token (e.g. USDC = 3e-5 btc) 
            - x: Collateral token (e.g. BTC)
            - LTV0: inital loan to value at time 0 
            - bs_vol: black shoral option volatilty 
            - Growth_rate: linear growth slope of y price 
            - Real_vol: growth Brownie motion volatility 
       
        Output: 
            - A dataframe with columns: ['t', 'y_price', 'borrowed_y', 'ltv', 'x_locked', 
            'y_locked', 'collateral_x', 'w_ytoken', 'PVCR'] 
        
    """
    # loan life duation between t(i) - t(i+1)
    delta_t = t[:-1]-t[1:]
    
    # token price percent change following a linear growth curve plus a random walk with volatility = Real_vol 
    rw = np.random.random(size = len(t)-1)
    delta_price = delta_t * (Growth_rate - 0.5*Real_vol**2) +  Real_vol *np.sqrt(delta_t)*np.random.normal(0, 1, len(t)-1) #norm.ppf(rw)
    
    # calcualte price with exponetial of cumulative sum of log price differences.  
    price_logdiff= np.hstack([0, delta_price]).cumsum()
    price = y_price*np.exp(price_logdiff)
    
    # reblance weights by black scholes option equestion. 
    w_y = norm.cdf((price_logdiff[:-1] + t[:-1]*(0.5*bs_vol**2))/bs_vol/np.sqrt(t[:-1]))
    
    # set a boundary [1e-8, 1-1e-8] to avoid 0 as denominator 
    w_y[w_y< 1e-8] = 1e-8   
    w_y[w_y> (1- 1e-8)] = 1- 1e-8
    
    # borrowed usd values in btc, depends on inital Collateral x, initial LTV0, initial y price and current price   
    borrowed_usd_btc = np.hstack([x*LTV0, x*LTV0*np.cumprod(price[1:]/price[:-1])]) 
    # Theoratical btc locked size given pool value, weigth and current price. 
    y_lock_change = w_y[1:] + (1-w_y[:-1])*price[:-2]/price[1:-1]*w_y[1:]/w_y[:-1]
    y_locked = np.hstack([x/y_price*w_y[0],x/y_price*w_y[0]*y_lock_change.cumprod()])
    # Theoratical usd locked size given pool value, weigth and current price. 
    x_locked = y_locked * (1-w_y) * price[:-1]/ w_y 
    # updated collateral_btc with new price 
    collateral_x = np.hstack([x,y_locked * price[1:] + x_locked]) 
    # divergence loss is defined as current collateral in btc over the initial caollateral  
    holdvalue = x
    loss = collateral_x/holdvalue 
    # ltv is defined as current borrowed values over the current collateral value. LTV > 1 means loan is default. 
    ltv = borrowed_usd_btc/collateral_x 
    # formate the output file as a dataframe for downstream analysis 
    _temp= np.column_stack((t, price, borrowed_usd_btc, ltv, np.append(x_locked, x_locked[-1]),\
                           np.append(y_locked, y_locked[-1]), collateral_x, np.append(w_y, w_y[-1]), loss))
    out_vec = pd.DataFrame(_temp, columns = ['t', 'y_price', 'borrowed_y', 'ltv', 'x_locked', 'y_locked', 'collateral_x', 'w_ytoken', 'pvcr'])
    return out_vec 

In [3]:
class PoolEngine: 
    """
        pool engine class
        Attribute: 
            - x: token 1 (e.g BTC) balance
            - y: token 2 (e.g USDC) balance
            - w_x: weight of x 
        Method:
            - sp(): spot price of one y token in terms of x token
            - in-given-out(inTtype, amount): size of in given out amount 
            - out-given-in(inTtype, amount): size of our given in amount 
            - in_given_price(inTtype, p): size of in to bring spot price to p the price of out token as function of in token
            - step(new_wx, oracle_p, verbose = 0): attributes updated if weights or price changes 
        
    """
    def __init__(self, x, y, w_x):
        self.x = x
        self.y = y
        self.w_x = w_x 
        assert 0<self.w_x<1 , "weights must be in (0,1) !"
        
    def step(self, new_wx, oracle_p, verbose=0):
        assert  0<new_wx<1 , "weights must be in (0,1) !"
        self.w_x = new_wx
        spot_price = self.sp()
        if spot_price < oracle_p: 
            delta_x = self.in_given_price('x', oracle_p)
            delta_y = self.out_given_in('x', delta_x)
            if verbose == 1:
                print('delta_x={}, and delta_y={}, price ={}, actual_price={}'.\
                  format(delta_x, delta_y, 1/oracle_p, delta_y/delta_x))
            self.x, self.y = self.x+delta_x, self.y-delta_y 
        elif spot_price > oracle_p:
            delta_y = self.in_given_price('y',1/oracle_p)
            delta_x = self.out_given_in('y', delta_y)
            if verbose == 1: 
                print('delta_x={}, and delta_y={}, price={}, actual_price={}'.\
                  format(delta_x, delta_y, 1/oracle_p,delta_y/delta_x))
            self.x, self.y = self.x-delta_x, self.y+delta_y 
        else:
            delta_x = 0
        return np.array([self.x, self.y, new_wx, spot_price, oracle_p, delta_x])
        
    def sp(self):
        
        w_y = 1- self.w_x
        p = self.x*w_y/(self.y*self.w_x) 
        return p 
    
    def in_given_out(self, inTtype, amount):
        if inTtype == 'x':
            assert 0<= amount < self.y, 'out amount must be greater than 0 and less than total'
            ratio = (1-self.w_x)/self.w_x 
            Ain = self.x * ((self.y/(self.y -amount))**(ratio) -1)
        else: 
            assert 0<= amount < self.x, 'out amount must be greater than 0 and less than total'
            ratio = self.w_x/(1-self.w_x)  
            Ain = self.y * ((self.x/(self.x -amount))**(ratio) -1)
        return Ain 
    
    def out_given_in(self, inTtype, amount):
        assert amount >= 0, 'amount must be greater than 0'
        if inTtype == 'x':
            ratio = self.w_x / (1-self.w_x)
            Aout = self.y * (1- (self.x/(self.x+amount))**ratio) 
        else: 
            ratio = (1-self.w_x)/self.w_x  
            Aout = self.x * (1- (self.y/(self.y+amount))**ratio) 
        return Aout

    def in_given_price(self, inTtype, price): 
        assert price >0 , 'price must be greater than 0'
        if inTtype == 'x':
            sp_price = self.sp()
            Ain = self.x * ((price/sp_price)**(1-self.w_x) - 1) 
        else: 
            sp_price = 1/self.sp()
            Ain = self.y * ((price/sp_price)**self.w_x - 1) 
        return Ain 
    